In [17]:
!pip install imageio[ffmpeg]

DEPRECATION: Loading egg at /home/phvguimaraes/miniconda3/envs/rs-env/lib/python3.11/site-packages/SOMperf-0.2b0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.5/29.5 MB 3.1 MB/s eta 0:00:00m eta 0:00:010:00:01


In [18]:
import numpy as np
import pygad
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.colors import LinearSegmentedColormap
import os
import imageio
from tqdm import tqdm

In [4]:
# Create output directory if it doesn't exist
if not os.path.exists('output'):
    os.makedirs('output')
if not os.path.exists('output/heatmaps'):
    os.makedirs('output/heatmaps')

In [5]:
# Shaffer's N6 Function (F6)
def shaffer_n6(x, y):
    numerator = np.sin(np.sqrt(x**2 + y**2))**2 - 0.5
    denominator = (1 + 0.001 * (x**2 + y**2))**2
    return 0.5 - numerator / denominator

In [6]:
# Convert binary chromosome to phenotype (x, y values)
def binary_to_float(chromosome, gene_length=25, min_val=-100, max_val=100):
    # Split chromosome into two genes (x and y)
    x_gene = chromosome[:gene_length]
    y_gene = chromosome[gene_length:]
    
    # Convert binary to decimal
    x_int = int(''.join(map(str, x_gene)), 2)
    y_int = int(''.join(map(str, y_gene)), 2)
    
    # Map to [-100, 100] range with 5 decimal precision
    scale = (max_val - min_val) / (2**gene_length - 1)
    x = min_val + x_int * scale
    y = min_val + y_int * scale
    
    return x, y

In [7]:
# Fitness function
def fitness_func(ga_instance, solution, solution_idx):
    x, y = binary_to_float(solution)
    # We add 1 to ensure positive fitness (Shaffer's N6 ranges ~ [0, 1])
    fitness = shaffer_n6(x, y) + 1
    return fitness

In [8]:
# Parameters
num_genes = 50  # 25 bits for x and 25 bits for y
gene_length = 25
num_generations = 500
population_size = 500
crossover_rate = 0.8
mutation_rate = 0.01
num_experiments = 32
save_interval = 10  # Save every 10 generations

# Store results from all experiments
all_best_fitness = np.zeros((num_experiments, num_generations))
all_avg_fitness = np.zeros((num_experiments, num_generations))
all_diversity = np.zeros((num_experiments, num_generations))  # Hamming distance as diversity measure

In [9]:
# Custom callback to track best individuals at intervals
class CustomCallback(pygad.GA):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.best_individuals = {}
        self.current_experiment = 0
    
    def on_generation(self, ga_instance):
        # Track best fitness and diversity
        generation = ga_instance.generations_completed
        all_best_fitness[self.current_experiment, generation-1] = ga_instance.best_solution()[1]
        all_avg_fitness[self.current_experiment, generation-1] = np.mean(ga_instance.last_generation_fitness)
        
        # Calculate population diversity (average Hamming distance)
        pop = ga_instance.population
        diversity = 0
        count = 0
        for i in range(len(pop)):
            for j in range(i+1, len(pop)):
                diversity += np.sum(pop[i] != pop[j])
                count += 1
        if count > 0:
            diversity /= count
        all_diversity[self.current_experiment, generation-1] = diversity
        
        # Save best individual at specified intervals
        if generation == 1 or generation % save_interval == 0:
            best_solution = ga_instance.best_solution()[0]
            self.best_individuals[generation] = best_solution.copy()
            
            # Save heatmap with individual
            if generation <= 100 or generation % 50 == 0:  # Save more frequently early on
                save_heatmap_with_individual(best_solution, generation, self.current_experiment)

In [10]:
def save_heatmap_with_individual(solution, generation, experiment_num):
    # Create heatmap of the function
    x_vals = np.linspace(-100, 100, 200)
    y_vals = np.linspace(-100, 100, 200)
    X, Y = np.meshgrid(x_vals, y_vals)
    Z = shaffer_n6(X, Y)
    
    # Get solution coordinates
    x, y = binary_to_float(solution)
    
    # Create plot
    plt.figure(figsize=(10, 8))
    plt.imshow(Z, extent=[-100, 100, -100, 100], origin='lower', cmap='viridis', alpha=0.8)
    plt.colorbar(label='Function Value')
    plt.scatter(x, y, c='red', s=100, label=f'Best Individual Gen {generation}')
    plt.title(f"Shaffer's N6 Function - Experiment {experiment_num+1}\nGeneration {generation}")
    plt.xlabel('x')
    plt.ylabel('y')
    plt.legend()
    
    # Save plot
    filename = f'output/heatmaps/exp_{experiment_num+1}_gen_{generation}.png'
    plt.savefig(filename, dpi=100, bbox_inches='tight')
    plt.close()

In [15]:
# Run experiments
for exp in range(num_experiments):
    print(f"\nRunning experiment {exp+1}/{num_experiments}")
    
    # Initialize GA with custom callback
    ga_instance = CustomCallback(
        num_generations=num_generations,
        num_parents_mating=int(population_size * 0.2),  # Keep 20% as parents
        fitness_func=fitness_func,
        sol_per_pop=population_size,
        num_genes=num_genes,
        gene_type=int,
        gene_space=[0, 1] * num_genes,
        parent_selection_type="rws",  # Roulette Wheel Selection (Fitness Proportionate)
        keep_parents=1,
        crossover_type="single_point",
        mutation_type="random",
        mutation_percent_genes=mutation_rate * 100,
        crossover_probability=crossover_rate,
        suppress_warnings=True,
        save_solutions=False
    )
    
    ga_instance.current_experiment = exp
    
    # Initialize population evenly distributed in the domain space
    initial_population = np.zeros((population_size, num_genes))
    for i in range(population_size):
        x_val = -100 + (i % 20) * 10
        y_val = -100 + (i // 20) * 10
    
        scale = (2**gene_length - 1) / 200
        x_int = min(int((x_val + 100) * scale), 2**gene_length - 1)
        y_int = min(int((y_val + 100) * scale), 2**gene_length - 1)
    
        x_gene = [int(b) for b in format(x_int, f'0{gene_length}b')][-gene_length:]
        y_gene = [int(b) for b in format(y_int, f'0{gene_length}b')][-gene_length:]
    
    initial_population[i] = x_gene + y_gene
    
    ga_instance.initial_population = initial_population
    
    # Run the GA
    ga_instance.run()
    
    # Save best individuals' genes for bit visualization
    if exp == np.argmax([np.max(all_best_fitness[i]) for i in range(num_experiments)]):
        best_exp_individuals = ga_instance.best_individuals
        best_exp_num = exp


Running experiment 1/32

Running experiment 2/32

Running experiment 3/32

Running experiment 4/32

Running experiment 5/32

Running experiment 6/32

Running experiment 7/32

Running experiment 8/32

Running experiment 9/32

Running experiment 10/32

Running experiment 11/32

Running experiment 12/32

Running experiment 13/32

Running experiment 14/32

Running experiment 15/32

Running experiment 16/32

Running experiment 17/32

Running experiment 18/32

Running experiment 19/32

Running experiment 20/32

Running experiment 21/32

Running experiment 22/32

Running experiment 23/32

Running experiment 24/32

Running experiment 25/32

Running experiment 26/32

Running experiment 27/32

Running experiment 28/32

Running experiment 29/32

Running experiment 30/32

Running experiment 31/32

Running experiment 32/32


In [22]:
# Create video from heatmaps of the best experiment
# Replace the video creation section with this improved version:

print("\nCreating heatmap video...")
heatmap_files = []
for gen in sorted(best_exp_individuals.keys()):
    heatmap_files.append(f'output/heatmaps/exp_{best_exp_num+1}_gen_{gen}.png')

try:
    # Try with imageio first
    with imageio.get_writer('output/best_experiment_evolution.mp4', fps=10) as writer:
        for filename in heatmap_files:
            image = imageio.imread(filename)
            writer.append_data(image)
except Exception as e:
    print(f"ImageIO MP4 creation failed: {e}")
    print("Trying alternative method with matplotlib...")
    
    # Create animation with matplotlib as fallback
    fig, ax = plt.subplots(figsize=(10, 8))
    im = ax.imshow(np.zeros((200, 200)), extent=[-100, 100, -100, 100], origin='lower', cmap='viridis')
    plt.colorbar(im, label='Function Value')
    point, = ax.plot([], [], 'ro', markersize=10)
    title = ax.set_title('')
    
    def update(frame):
        image = plt.imread(heatmap_files[frame])
        im.set_array(image[:, :, :3])  # Remove alpha channel if present
        x, y = binary_to_float(best_exp_individuals[sorted(best_exp_individuals.keys())[frame]])
        point.set_data([x], [y])
        title.set_text(f'Generation {sorted(best_exp_individuals.keys())[frame]}')
        return im, point, title
    
    ani = FuncAnimation(fig, update, frames=len(heatmap_files), interval=100, blit=True)
    ani.save('output/best_experiment_evolution.mp4', writer='ffmpeg', fps=10)
    plt.close()


Creating heatmap video...


In [21]:
# Plot gene bits for best experiment
print("Creating gene bits visualization...")
generations = sorted(best_exp_individuals.keys())
bit_data = np.zeros((len(generations), num_genes))

for i, gen in enumerate(generations):
    bit_data[i] = best_exp_individuals[gen]

plt.figure(figsize=(12, 8))
plt.imshow(bit_data, cmap='binary', aspect='auto', interpolation='none')
plt.colorbar(label='Bit Value (0/1)')
plt.yticks(range(len(generations)), generations)
plt.title('Gene Bits Evolution - Best Experiment')
plt.xlabel('Bit Position')
plt.ylabel('Generation')
plt.savefig('output/gene_bits_evolution.png', dpi=150, bbox_inches='tight')
plt.close()

Creating gene bits visualization...


/tmp/ipykernel_41309/3613908556.py:10: UserWarning: Attempting to set identical low and high ylims makes transformation singular; automatically expanding.
  plt.imshow(bit_data, cmap='binary', aspect='auto', interpolation='none')


In [23]:
# Plot fitness and diversity across experiments
print("Creating convergence and diversity plots...")
avg_best_fitness = np.mean(all_best_fitness, axis=0)
std_best_fitness = np.std(all_best_fitness, axis=0)

avg_avg_fitness = np.mean(all_avg_fitness, axis=0)
std_avg_fitness = np.std(all_avg_fitness, axis=0)

avg_diversity = np.mean(all_diversity, axis=0)
std_diversity = np.std(all_diversity, axis=0)

generations = range(1, num_generations + 1)

plt.figure(figsize=(12, 6))

plt.fill_between(generations, 
                 avg_best_fitness - std_best_fitness/2, 
                 avg_best_fitness + std_best_fitness/2, 
                 color='blue', alpha=0.2)

plt.fill_between(generations, 
                 avg_diversity - std_diversity/2, 
                 avg_diversity + std_diversity/2, 
                 color='red', alpha=0.2)

plt.plot(generations, avg_best_fitness, 'b-', label='Average Best Fitness (Convergence)')
plt.plot(generations, avg_diversity, 'r-', label='Average Population Diversity')
plt.xlabel('Generation')
plt.ylabel('Value')
plt.title('Convergence and Diversity Across 32 Experiments')
plt.legend()
plt.grid(True)
plt.savefig('output/convergence_diversity.png', dpi=150, bbox_inches='tight')
plt.close()

Creating convergence and diversity plots...
